---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    def decider(g):
        avg_clustering = nx.average_clustering(g)
        degrees = set(g.degree().values())
        if len(degrees) > 10: # i.e. more than 10 different degrees occur in the network
            return 'PA'
        if avg_clustering < 0.1: # high probability -> low clustering
            return 'SW_H'
        return 'SW_L'
    
    return list(map(decider, P1_Graphs))

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_prediction_features():
    df = pd.DataFrame(index=G.nodes())
    df['degree'] = pd.Series(G.degree())
    df['management_salary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    return df
    
salary_prediction_features().head()

,degree,management_salary
0,44,0.0
1,52,NaN
2,95,NaN
3,71,1.0
4,96,1.0


In [29]:
from sklearn.linear_model import LogisticRegression
def salary_predictions():
    feature_matrix = salary_prediction_features()
    training_data = feature_matrix[~pd.isnull(feature_matrix['management_salary'])]

    data_to_predict = feature_matrix[pd.isnull(feature_matrix['management_salary'])].drop('management_salary', axis=1)
    
    X_train = training_data.drop('management_salary', axis=1)
    y_train = training_data['management_salary']
    
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    probabilities = clf.predict_proba(data_to_predict)[:, 1]
    return pd.Series(probabilities,data_to_predict.index)

salary_predictions()

1       0.204648
2       0.568012
5       0.959179
8       0.127135
14      0.426709
18      0.230454
27      0.280860
30      0.320708
31      0.192575
34      0.103944
37      0.127135
40      0.258456
45      0.181052
54      0.198542
55      0.154607
60      0.223795
62      0.992339
65      0.530463
77      0.084574
79      0.159630
97      0.084574
101     0.097087
103     0.204648
108     0.154607
113     0.492565
122     0.090636
141     0.454753
142     0.882756
144     0.059462
145     0.345990
          ...   
913     0.073542
914     0.078882
915     0.035839
918     0.093812
923     0.043002
926     0.059462
931     0.063849
934     0.039986
939     0.038555
944     0.035839
945     0.041468
947     0.049699
950     0.164785
951     0.051521
953     0.061619
959     0.035839
962     0.037173
963     0.149713
968     0.039986
969     0.037173
974     0.044591
984     0.038555
987     0.053407
989     0.039986
991     0.039986
992     0.038555
994     0.035839
996     0.0358

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [22]:
def new_connections_features():
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    
    df['pa'] = [x[2] for x in preferential_attachment]
    df['rai'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jc'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    df = df.join(future_connections)
    
    return df
    
new_connections_features().head()

,pa,rai,jc,Future Connection
"(0, 2)",4180,0.055340,0.045802,0.0
"(0, 3)",3124,0.021388,0.027273,0.0
"(0, 4)",4224,0.021388,0.022222,0.0
"(0, 7)",3168,0.061668,0.036364,0.0
"(0, 8)",1628,0.011628,0.012821,0.0


In [23]:
from sklearn.linear_model import LogisticRegression
def new_connections_predictions():
    feature_matrix = new_connections_features()
    training_data = feature_matrix[~pd.isnull(feature_matrix['Future Connection'])]

    data_to_predict = feature_matrix[pd.isnull(feature_matrix['Future Connection'])].drop('Future Connection', axis=1)
    
    X_train = training_data.drop('Future Connection', axis=1)
    y_train = training_data['Future Connection']
    
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    probabilities = clf.predict_proba(data_to_predict)[:, 1]
    return pd.Series(probabilities,data_to_predict.index)

new_connections_predictions()

(0, 9)          0.060240
(0, 19)         0.137690
(0, 20)         0.324420
(0, 35)         0.026414
(0, 38)         0.017572
(0, 42)         0.720990
(0, 43)         0.025063
(0, 50)         0.014063
(0, 53)         0.674097
(0, 54)         0.080681
(0, 62)         0.599228
(0, 63)         0.126893
(0, 69)         0.036863
(0, 72)         0.013153
(0, 83)         0.409815
(0, 90)         0.040514
(0, 91)         0.023348
(0, 95)         0.161154
(0, 100)        0.058605
(0, 106)        0.939438
(0, 108)        0.024459
(0, 109)        0.012862
(0, 110)        0.013450
(0, 118)        0.015722
(0, 122)        0.045986
(0, 131)        0.084803
(0, 133)        0.824684
(0, 140)        0.076967
(0, 149)        0.350091
(0, 151)        0.017442
                  ...   
(988, 989)      0.012409
(988, 996)      0.012396
(988, 997)      0.816802
(988, 1000)     0.012417
(988, 1002)     0.012396
(989, 994)      0.012400
(989, 996)      0.012400
(989, 1003)     0.012400
(989, 1004)     0.012400
